### TOOLS + CHAINS ==> LangChain AGENTS
        - you have one big chain for task A [Summarization]
        - you have one big chain for task B [Translation]
        - you have one big chain for task C [Classification]
        - In order to make the system more flexible, can use all these we i9ntroduce AGENTS

In [1]:
import yaml, os, openai
from langchain.chains import LLMChain
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [2]:
# !pip install google-search-results

In [3]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
os.environ['SERPAPI_API_KEY'] = credentials['SERPAPI_API_KEY']
os.environ['ENGINE'] = credentials['ENGINE']

openai.api_key = credentials['OPENAI_API_KEY']
openai.api_base = credentials['OPENAI_API_BASE']
openai.api_type = credentials['OPENAI_API_TYPE']
openai.api_version = credentials['OPENAI_API_VERSION']
openai.engine = credentials['ENGINE']

In [4]:
llm = ChatOpenAI(
                openai_api_key=os.environ["OPENAI_API_KEY"],
                engine = os.environ["ENGINE"],
                model='gpt-3.5-turbo',
                temperature=0.9, 
                max_tokens = 256
                )

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [5]:
tools = load_tools(
                    ["serpapi", "llm-math"],  # we loaded 2 tools here
                    llm=llm
                    )

In [6]:
# let's explore about llm-math chain
tools[1].name, tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [7]:
tools[1]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```outpu

### Initialize the Agent

In [8]:
agent = initialize_agent(
                        tools, 
                        llm, 
                        agent="zero-shot-react-description", 
                        verbose=True
                        )

In [9]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [14]:
## Standard LLM Query
agent.run("Hi How are you today?")



> Entering new AgentExecutor chain...
This question doesn't require any tools to answer. It is a simple greeting and does not require a search engine or calculator.
Final Answer: No answer needed as it is a greeting.

> Finished chain.


'No answer needed as it is a greeting.'

In [11]:
agent.run("Who is the United States President? What is his current age raised divided by 2?")



> Entering new AgentExecutor chain...
I can use a search engine to find the current United States President. Then I can calculate his current age divided by 2.
Action: Search
Action Input: "current United States President"
Observation: Joe Biden
Thought:Now I need to find Joe Biden's current age.
Action: Search
Action Input: "Joe Biden age"
Observation: 80 years
Thought:Now I can calculate his current age divided by 2.
Action: Calculator
Action Input: 80 / 2
Observation: Answer: 40.0
Thought:I now know the final answer.
Final Answer: Joe Biden's current age raised divided by 2 is 40.0.

> Finished chain.


"Joe Biden's current age raised divided by 2 is 40.0."

In [15]:
agent.run("What is the average age in the United States? How much less is that then the age of the current US President?")



> Entering new AgentExecutor chain...
I need to find the average age in the United States and the age of the current US President. Then I can calculate the difference between the two.
Action: Search
Action Input: "average age in the United States"
Observation: ["The U.S. is a country of 50 states covering a vast swath of North America, with Alaska in the northwest and Hawaii extending the nation’s presence into the Pacific Ocean. Major Atlantic Coast cities are New York, a global finance and culture center, and capital Washington, DC. Midwestern metropolis Chicago is known for influential architecture and on the west coast, Los Angeles' Hollywood is famed for filmmaking. ― Google", 'United States type: Country in North America.', 'United States kgmid: /m/09c7w0.', 'United States capital: Washington, D.C..', 'United States co2_emissions_per_capita: 14.67 metric tons (2019).', 'United States electricity_consumption_per_capita: 12,993.97 kWh (2014).', 'United States energy_use_per_capit

'The average age in the United States is 38.9 years. The current US President is 80 years old. The difference between the two is 41.1 years.'

# more complex agent

In [19]:
# !pip install wikipedia

In [20]:
tools = load_tools(
                    ["serpapi", "llm-math","wikipedia","terminal"], 
                    llm=llm
                    )
agent = initialize_agent(
                        tools, 
                        llm, 
                        agent="zero-shot-react-description", 
                        verbose=True
                        )

In [22]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.
Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
terminal: Run shell commands on this MacOS machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, Wikipedia, terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!



In [23]:
agent.run("Who is the head of DeepMind")



> Entering new AgentExecutor chain...
I need to find out the name of the head of DeepMind.
Action: Wikipedia
Action Input: "DeepMind"
Observation: Page: Google DeepMind
Summary: DeepMind Technologies Limited, doing business as Google DeepMind, is a British-American artificial intelligence research laboratory which serves as a subsidiary of Google. Founded in the UK in 2010, it was acquired by Google in 2014, becoming a wholly owned subsidiary of Google parent company Alphabet Inc. after Google's corporate restructuring in 2015. The company is based in London, with research centres in Canada, France, and the United States.
Google DeepMind has created neural network models that learn how to play video games in a fashion similar to that of humans, as well as Neural Turing machines (neural networks that can access external memory like a conventional Turing machine),, resulting in a computer that loosely resembles short-term memory in the human brain.DeepMind made headlines in 2016 after 

'Demis Hassabis'

In [25]:
agent.run("What is DeepMind ?")



> Entering new AgentExecutor chain...
I'm not exactly sure what DeepMind is, so I think I should start by searching for information about it.
Action: Search
Action Input: "DeepMind"
Observation: [{'title': 'Google DeepMind unites researchers in bid to create an ImageNet of robot actions', 'link': 'https://techcrunch.com/2023/10/05/google-deepmind-unites-researchers-in-bid-to-create-an-imagenet-of-robot-actions/', 'source': 'TechCrunch', 'date': '13 hours ago', 'thumbnail': 'https://serpapi.com/searches/651fa41c8e02e4e86f2da72c/images/7619d063c6d718997eb2ef6ff28e93a23645a2f292515359.jpeg'}, {'title': "DeepMind cofounder is tired of 'knee-jerk bad takes' about AI", 'link': 'https://venturebeat.com/ai/deepmind-cofounder-is-tired-of-knee-jerk-bad-takes-about-ai/', 'source': 'VentureBeat', 'date': '16 hours ago', 'thumbnail': 'https://serpapi.com/searches/651fa41c8e02e4e86f2da72c/images/7619d063c6d718991fb90b700486c6b3d4692be5a5fb410e.jpeg'}, {'title': 'Scaling up learning across many dif

'DeepMind is an AI research laboratory that was acquired by Google. It has made notable contributions in various areas of AI and has developed programs like AlphaGo.'

In [26]:
agent.run("Take the year DeepMind was founded and add 50 years. Will this year have AGI?")



> Entering new AgentExecutor chain...
I need to find out the year DeepMind was founded and add 50 years to it. Then I can determine if this year will have AGI.
Action: Search
Action Input: "DeepMind founding year"
Observation: 2010, London, United Kingdom
Thought:I have found the founding year of DeepMind.
Action: Calculator
Action Input: 2010 + 50
Observation: Answer: 2060
Thought:I now know the final answer.
Final Answer: The year DeepMind was founded plus 50 years is 2060. Whether or not this year will have AGI is unknown.

> Finished chain.


'The year DeepMind was founded plus 50 years is 2060. Whether or not this year will have AGI is unknown.'

In [28]:
agent.run("What files are in my current directory ?")



> Entering new AgentExecutor chain...
I can use the terminal to list the files in the current directory.
Action: terminal
Action Input: ls
Observation: 1 langchain-intro.ipynb
12 langchain talk to csv.ipynb
2 langchain-tools-and-chains.ipynb
3 chatgpt-langchain-integration.ipynb
4 conversation with memory.ipynb
5 langchain-meets-huggingface.ipynb
6 use-hf-models-local.ipynb
7 summarization system.ipynb
8 langchain agents.ipynb
assets
cadentials.yaml
data
image.png

Thought:I have the list of files in the current directory.
Final Answer: The files in the current directory are: langchain-intro.ipynb, langchain talk to csv.ipynb, langchain-tools-and-chains.ipynb, chatgpt-langchain-integration.ipynb, conversation with memory.ipynb, langchain-meets-huggingface.ipynb, use-hf-models-local.ipynb, summarization system.ipynb, langchain agents.ipynb, assets, credentials.yaml, data, and image.png.

> Finished chain.


'The files in the current directory are: langchain-intro.ipynb, langchain talk to csv.ipynb, langchain-tools-and-chains.ipynb, chatgpt-langchain-integration.ipynb, conversation with memory.ipynb, langchain-meets-huggingface.ipynb, use-hf-models-local.ipynb, summarization system.ipynb, langchain agents.ipynb, assets, credentials.yaml, data, and image.png.'